In [86]:
!pip install -q twarc
!pip install -q jsonlines

In [111]:
import pandas as pd
import twarc

### Opening different IEEE datasets and meging them into one dataset

In [112]:
data1=pd.read_csv("a.csv")
data2=pd.read_csv("b.csv")
data3=pd.read_csv("c.csv")
data4=pd.read_csv("d.csv")
data5=pd.read_csv("e.csv")

In [113]:
data1.columns=["id","sentiment_score"]
data2.columns=["id","sentiment_score"]
data3.columns=["id","sentiment_score"]
data4.columns=["id","sentiment_score"]
data5.columns=["id","sentiment_score"]
f=[data1,data2,data3,data4,data5]
data=pd.concat(f)
data.shape

(5962, 2)

In [114]:
data.head()

,id,sentiment_score
0,1251721949899665409,0.166667
1,1251722077851324420,0.305455
2,1251722077905788929,0.154167
3,1251722190795481093,0.283333
4,1251722436741083136,0.000000


## Hydrating tweets from dataset

In [115]:
import tweepy
# Insert API Keys here { run : "auto"}

# These keys are received after applying for a twitter developer account

consumer_key = "rm2bLDjA2BzljoA0GomL5o6W7"
consumer_secret = "xiFBG4VKWPuQts1v3uqAesllpDp36y44YkFnzBtezSbSYW9dBV"
access_token = "935519854064418816-sOBxmFMaDygAx3FQXRBjH0drpZ2OXpB"
access_token_secret = "GbOTefzapdet9vpmR3H9OBRuJNJNs1cI4Adh5HrkIYPJz"

t = twarc.Twarc(consumer_key, consumer_secret, access_token, access_token_secret) #Initializing Twarc

In [116]:
import jsonlines, json
data1 = data["id"] #Taking only Tweet IDs for Hydration
ids = data1.values.tolist() #Getting list of tweet ids from pandas DataFrame
hydrated_tweets = [] #Creating empty list
ids_to_hydrate = set(ids) #Creating ids_to_hydrate list

# Now, use twarc and start hydrating
for tweet in t.hydrate(ids_to_hydrate): #Initializing Hydrate Iterator with twarc
  if tweet['place'] != None: #Checking for place value
    if tweet['place']['country'] == 'India': #Checking if Country is India
      hydrated_tweets.append(tweet) #Appending Obtained tweet to hydrated_tweets list

In [117]:
output=pd.DataFrame.from_dict(hydrated_tweets, orient='columns')

### Merging hydrated tweets with original dataframe

In [118]:
data = pd.merge(output,data,on='id')
data.head()

,created_at,id,id_str,full_text,truncated,display_text_range,entities,source,in_reply_to_status_id,in_reply_to_status_id_str,...,favorited,retweeted,possibly_sensitive,lang,quoted_status_id,quoted_status_id_str,quoted_status_permalink,quoted_status,extended_entities,sentiment_score
0,Fri May 08 16:04:47 +0000 2020,1258789958229340160,1258789958229340160,We are doing everything in our power to *Fight...,False,"[0, 237]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://instagram.com"" rel=""nofollow"">...",NaN,None,...,False,False,False,en,NaN,NaN,NaN,NaN,NaN,-0.003571
1,Fri May 08 17:23:47 +0000 2020,1258809838727036946,1258809838727036946,Adhering to guidelines and timings given by ou...,False,"[0, 240]","{'hashtags': [{'text': 'COVID19', 'indices': [...","<a href=""http://instagram.com"" rel=""nofollow"">...",NaN,None,...,False,False,True,en,NaN,NaN,NaN,NaN,NaN,-0.100000
2,Thu May 14 12:16:25 +0000 2020,1260906811454095375,1260906811454095375,List of today's Covid19 patients\n\n#IndiaFigh...,False,"[0, 240]","{'hashtags': [{'text': 'IndiaFightsCorona', 'i...","<a href=""http://instagram.com"" rel=""nofollow"">...",NaN,None,...,False,False,True,en,NaN,NaN,NaN,NaN,NaN,0.000000
3,Thu May 14 17:35:26 +0000 2020,1260987098045317121,1260987098045317121,Wows their believed reopen like coronavirus ca...,False,"[0, 237]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://instagram.com"" rel=""nofollow"">...",NaN,None,...,False,False,False,en,NaN,NaN,NaN,NaN,NaN,0.000000
4,Sun Apr 19 05:41:47 +0000 2020,1251747804155973632,1251747804155973632,The decision of PM Narendra Modi to extend the...,False,"[0, 238]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://instagram.com"" rel=""nofollow"">...",NaN,None,...,False,False,False,en,NaN,NaN,NaN,NaN,NaN,0.000000


### selecting only important features

In [119]:
data.drop_duplicates(subset ="id", keep = False, inplace = True)#removing duplicates
data= data[['id','retweet_count','lang','full_text','sentiment_score']]#selection of Features
data = data[data["lang"]=="en"]#Extracting the tweets from dataframe whose language is English

In [120]:
data.head()

,id,retweet_count,lang,full_text,sentiment_score
0,1258789958229340160,0,en,We are doing everything in our power to *Fight...,-0.003571
1,1258809838727036946,0,en,Adhering to guidelines and timings given by ou...,-0.100000
2,1260906811454095375,0,en,List of today's Covid19 patients\n\n#IndiaFigh...,0.000000
3,1260987098045317121,0,en,Wows their believed reopen like coronavirus ca...,0.000000
4,1251747804155973632,0,en,The decision of PM Narendra Modi to extend the...,0.000000


In [121]:
data.dtypes

id                   int64
retweet_count        int64
lang                object
full_text           object
sentiment_score    float64
dtype: object

### Adding sentiments

In [122]:
# Function to Convert the IEEE Sentiment score to Sentiment Text namely positive, negative, neutral
def func(x):
    if x < 0:
        return "negative"
    elif x == 0:
        return "neutral"
    else:
        return "positive"
data['sentiment_score'] = data['sentiment_score'].apply(lambda x : func(x))#converting sentiment score to sentiment text

In [123]:
data.rename(columns={"sentiment_score":"sentiment"},inplace=True)
data.head()

,id,retweet_count,lang,full_text,sentiment
0,1258789958229340160,0,en,We are doing everything in our power to *Fight...,negative
1,1258809838727036946,0,en,Adhering to guidelines and timings given by ou...,negative
2,1260906811454095375,0,en,List of today's Covid19 patients\n\n#IndiaFigh...,neutral
3,1260987098045317121,0,en,Wows their believed reopen like coronavirus ca...,neutral
4,1251747804155973632,0,en,The decision of PM Narendra Modi to extend the...,neutral


## Cleaning of tweets

In [125]:
#Defining functions for Cleaning and Normalization of Data
import re
import string
def replace_url(string): # cleaning of URL
    text = re.sub(r'http\S+', 'LINK', string)
    return text


def replace_email(text):#Cleaning of Email related text
    line = re.sub(r'[\w\.-]+@[\w\.-]+','MAIL',str(text))
    return "".join(line)

def rep(text):#cleaning of non standard words
    grp = text.group(0)
    if len(grp) > 3:
        return grp[0:2]
    else:
        return grp# can change the value here on repetition
def unique_char(rep,sentence):
    convert = re.sub(r'(\w)\1+', rep, sentence) 
    return convert

def find_dollar(text):#Finding the dollar sign in the text
    line=re.sub(r'\$\d+(?:\.\d+)?','PRICE',text)
    return "".join(line)

def replace_emoji(text):
    emoji_pattern = re.compile("["
    u"\U0001F600-\U0001F64F" # emoticons
    u"\U0001F300-\U0001F5FF" # symbols & pictographs
    u"\U0001F680-\U0001F6FF" # transport & map symbols
    u"\U0001F1E0-\U0001F1FF" # flags (iOS)
    u"\U00002702-\U000027B0"
    u"\U000024C2-\U0001F251"
    "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'EMOJI', text) 
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']',
          '>', '%', '=', '#', '*', '+', '\\', '•', '~', '@', '£', '·', '_', '{', '}', '©', '^',
          '®', '`', '<', '→', '°', '€', '™', '›', '♥', '←', '×', '§', '″', '′', 'Â', '█',
          '½', 'à', '…', '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶',
          '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', '▒', '：', '¼',
          '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲',
          'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', '∙', '）', '↓', '、', '│', '（', '»', '，', '♪',
          '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√']

def clean_text(text: str) -> str:
    text = str(text)
    for punct in puncts + list(string.punctuation):
        if punct in text:
            text = text.replace(punct, f'')
    return text
   
def replace_asterisk(text):
    text = re.sub("\*", 'ABUSE ', text)
    return text

def remove_duplicates(text):
    text = re.sub(r'\b(\w+\s*)\1{1,}', '\\1', text)
    return text

def change(text):
    if(text == ''):
        return text
  #calling the subfunctions in the cleaning function
    text = replace_email(text)
    text = replace_url(text)
    text = unique_char(rep,text)
    text = replace_asterisk(text)
    text = remove_duplicates(text)
    text = clean_text(text)
    return text

In [126]:
data['full_text'] = data['full_text'].apply(lambda x : change(x)) # Running cleaning and normalization function on datasets

In [127]:
data.head()

,id,retweet_count,lang,full_text,sentiment
0,1258789958229340160,0,en,We are doing everything in our power to ABUSE ...,negative
1,1258809838727036946,0,en,Adhering to guidelines and timings given by ou...,negative
2,1260906811454095375,0,en,List of todays Covid19 patients\n\nIndiaFights...,neutral
3,1260987098045317121,0,en,Wows their believed reopen like coronavirus ca...,neutral
4,1251747804155973632,0,en,The decision of PM Narendra Modi to extend the...,neutral


In [ ]:
data.to_csv("Data.csv")